In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

print("GPU disponible:", tf.config.list_physical_devices('GPU'))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Tamaño y batch
img_size = (48, 48)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

# Ruta del dataset
dataDir = "dataset/train"

# Carga de datos
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataDir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

original_train_ds = tf.keras.utils.image_dataset_from_directory(
    dataDir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

# Clases
class_names = original_train_ds.class_names

# Normalización
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# Pipeline optimizado
train_ds = original_train_ds.cache().shuffle(1000).map(
    lambda x, y: (data_augmentation(normalization_layer(x), training=True), y),
    num_parallel_calls=AUTOTUNE
).prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.cache().map(
    lambda x, y: (normalization_layer(x), y),
    num_parallel_calls=AUTOTUNE
).prefetch(buffer_size=AUTOTUNE)

# Verifica GPU
print("Dispositivos GPU disponibles:", tf.config.list_physical_devices('GPU'))


## Entrenamiento:

In [ ]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

train_ds = train_ds.map(lambda x, y: (data_augmentation(x,training=True), y))

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

model = models.Sequential([
    keras.layers.Input(shape=(48, 48, 1)),

    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)


y_train = []
for _, labels in train_ds.unbatch():
    y_train.append(labels.numpy())
y_train = np.array(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))

# Callback para detener entrenamiento si no mejora
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)

# Callback para guardar el mejor modelo
checkpoint = ModelCheckpoint(
    "best_model.h5",          # nombre del archivo
    monitor='val_loss',       # métrica a vigilar
    save_best_only=True,      # solo guarda si mejora
    verbose=1
)

# Entrenamiento con callbacks
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early_stop, checkpoint],  # aquí incluyes ambos
    class_weight=class_weights_dict
)


## Descargar el modelo:
ya que lo estaba haciendo directamente desde google colab, con esto descargo el modelo de la pagina.

In [23]:
from google.colab import files
files.download("best_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Matriz de confusion:

In [15]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predicciones
y_pred = np.argmax(model.predict(val_ds), axis=1)

# Etiquetas verdaderas
y_true = []
for _, labels in val_ds.unbatch():
    y_true.append(labels.numpy())
y_true = np.array(y_true)

# Reporte
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))



180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step
[[397  93  22  62 233  52]
 [180 171   9  61 190 199]
 [187  41 894  94 106  86]
 [ 58  89  57 472 312  53]
 [170 123  29 133 486  37]
 [ 24  55  13  16  12 525]]
              precision    recall  f1-score   support

       angry       0.39      0.46      0.42       859
        fear       0.30      0.21      0.25       810
       happy       0.87      0.63      0.74      1408
     neutral       0.56      0.45      0.50      1041
         sad       0.36      0.50      0.42       978
    surprise       0.55      0.81      0.66       645

    accuracy                           0.51      5741
   macro avg       0.51      0.51      0.50      5741
weighted avg       0.54      0.51      0.52      5741



In [16]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Matriz de Confusión')
plt.show()


ModuleNotFoundError: No module named 'seaborn'

## Evaluacion del accuaracy

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    image_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size
)

test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)


test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.2f}")


## Grafica para ver la accuaracy del modelo:


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()


## Guardar el modelo

In [ ]:
model.save("emotion_model.h5")
# model.save("emotion_model.keras") <-- Esta forma la recomienda keras ya que el que usamos se ve mas antiguo, pero ocupo confirmar con la documentación